# Optimization

## Preprocessing

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv('/content/drive/My Drive/UC Berkeley Data Analytics Bootcamp/Module 21/Challenge/charity_data_preprocessed.csv')
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### AFFILIATION

In [3]:
# Look at AFFILIATION value counts for binning
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [4]:
# Choose a cutoff value and create a list of classifications to be replaced

affiliations_to_replace = list(application_df['AFFILIATION'].value_counts().loc[lambda x: x < 15000].index)

# Replace in dataframe
for cls in affiliations_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

### USE_CASE

In [5]:
# Look at USE_CASE value counts for binning
application_df['USE_CASE'].value_counts()

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of classifications to be replaced

use_cases_to_replace = list(application_df['USE_CASE'].value_counts().loc[lambda x: x < 5000].index)

# Replace in dataframe
for cls in use_cases_to_replace:
    application_df['USE_CASE'] = application_df['USE_CASE'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['USE_CASE'].value_counts()

Preservation    28095
ProductDev       5671
Other             533
Name: USE_CASE, dtype: int64

### ORGANIZATION


In [7]:
# Look at USE_CASE value counts for binning
application_df['ORGANIZATION'].value_counts()

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced

organization_to_replace = list(application_df['ORGANIZATION'].value_counts().loc[lambda x: x < 10000].index)

# Replace in dataframe
for cls in organization_to_replace:
    application_df['ORGANIZATION'] = application_df['ORGANIZATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['ORGANIZATION'].value_counts()

Trust          23515
Association    10255
Other            529
Name: ORGANIZATION, dtype: int64

### STATUS

In [9]:
# Look at USE_CASE value counts for binning
application_df['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [10]:
# Drop STATUS
application_df = application_df.drop(columns=['STATUS'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Other,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Other,Trust,100000-499999,N,142590,1


### SPECIAL_CONSIDERATIONS

In [11]:
# Look at SPECIAL_CONSIDERATIONS value counts for binning
application_df['SPECIAL_CONSIDERATIONS'].value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [12]:
# Drop STATUS
application_df = application_df.drop(columns=['SPECIAL_CONSIDERATIONS'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Other,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Other,Trust,100000-499999,142590,1


# Split and Scale

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
X = application_df.drop(columns='IS_SUCCESSFUL')
X = pd.get_dummies(X)

# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# number of features
num_of_features = len(X_test_scaled[0])
num_of_features

34

## Keras Tuner

In [14]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first hidden layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=3,
        max_value=43,
        step=5), activation=activation, input_dim = num_of_features))

    # Allow kerastuner to decide number of extra hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i+1),
            min_value=3,
            max_value=43,
            step=5),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation=activation))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [15]:
# Import the kerastuner library
!pip install keras-tuner --upgrade
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 2.9 MB/s eta 0:00:00


In [16]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 53s]
val_accuracy: 0.7367929816246033

Best val_accuracy So Far: 0.7371428608894348
Total elapsed time: 00h 23m 53s


In [17]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'sigmoid', 'first_units': 33, 'num_layers': 2, 'units_0': 13, 'units_1': 3, 'units_2': 23, 'units_3': 18, 'units_4': 18, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0019'}
{'activation': 'sigmoid', 'first_units': 8, 'num_layers': 1, 'units_0': 23, 'units_1': 28, 'units_2': 23, 'units_3': 38, 'units_4': 33, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0015'}
{'activation': 'tanh', 'first_units': 23, 'num_layers': 2, 'units_0': 28, 'units_1': 28, 'units_2': 13, 'units_3': 23, 'units_4': 3, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}


In [18]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5630 - accuracy: 0.7371 - 1s/epoch - 4ms/step
Loss: 0.5630009770393372, Accuracy: 0.7371428608894348
268/268 - 2s - loss: 0.5496 - accuracy: 0.7370 - 2s/epoch - 7ms/step
Loss: 0.5495813488960266, Accuracy: 0.7370262145996094
268/268 - 2s - loss: 0.5530 - accuracy: 0.7369 - 2s/epoch - 7ms/step
Loss: 0.5530088543891907, Accuracy: 0.7369096279144287


# Compiling, Training and Evaluating the Model: TRIAL 1/3

In [19]:
top_hyper[0].values

{'activation': 'sigmoid',
 'first_units': 33,
 'num_layers': 2,
 'units_0': 13,
 'units_1': 3,
 'units_2': 23,
 'units_3': 18,
 'units_4': 18,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0019'}

In [20]:
top_hyper[0].values['first_units']

33

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[0].values['first_units'], activation = top_hyper[0].values['activation'], input_dim = num_of_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[0].values['units_0'], activation = top_hyper[0].values['activation']))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[0].values['units_1'], activation = top_hyper[0].values['activation']))

# Forth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[0].values['units_2'], activation = top_hyper[0].values['activation']))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[0].values['units_3'], activation = top_hyper[0].values['activation']))

# Sixth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[0].values['units_4'], activation = top_hyper[0].values['activation']))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = top_hyper[0].values['activation']))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 33)                1155      
                                                                 
 dense_5 (Dense)             (None, 13)                442       
                                                                 
 dense_6 (Dense)             (None, 3)                 42        
                                                                 
 dense_7 (Dense)             (None, 23)                92        
                                                                 
 dense_8 (Dense)             (None, 18)                432       
                                                                 
 dense_9 (Dense)             (None, 18)                342       
                                                                 
 dense_10 (Dense)            (None, 1)                

In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics="accuracy")

In [23]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=20)

Epoch 1/20
804/804 [==============================] - 7s 6ms/step - loss: 0.6500 - accuracy: 0.6273
Epoch 2/20
804/804 [==============================] - 4s 5ms/step - loss: 0.5788 - accuracy: 0.7282
Epoch 3/20
804/804 [==============================] - 5s 6ms/step - loss: 0.5776 - accuracy: 0.7280
Epoch 4/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5766 - accuracy: 0.7283
Epoch 5/20
804/804 [==============================] - 5s 6ms/step - loss: 0.5758 - accuracy: 0.7289
Epoch 6/20
804/804 [==============================] - 4s 5ms/step - loss: 0.5753 - accuracy: 0.7289
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5750 - accuracy: 0.7288
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5751 - accuracy: 0.7277
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5744 - accuracy: 0.7296
Epoch 10/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5739 - accuracy: 0.7286

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5496 - accuracy: 0.7342 - 498ms/epoch - 2ms/step
Loss: 0.5495648980140686, Accuracy: 0.7342274188995361


In [25]:
# Export our model to HDF5 file
nn.save_weights('AlphabetSoupCharity_Optimization_1.h5')

# Compiling, Training and Evaluating the Model: TRIAL 2/3

In [26]:
top_hyper[1].values

{'activation': 'sigmoid',
 'first_units': 8,
 'num_layers': 1,
 'units_0': 23,
 'units_1': 28,
 'units_2': 23,
 'units_3': 38,
 'units_4': 33,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0015'}

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[1].values['first_units'], activation = top_hyper[1].values['activation'], input_dim = num_of_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[1].values['units_0'], activation = top_hyper[1].values['activation']))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[1].values['units_1'], activation = top_hyper[1].values['activation']))

# Forth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[1].values['units_2'], activation = top_hyper[1].values['activation']))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[1].values['units_3'], activation = top_hyper[1].values['activation']))

# Sixth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[1].values['units_4'], activation = top_hyper[1].values['activation']))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = top_hyper[1].values['activation']))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 8)                 280       
                                                                 
 dense_12 (Dense)            (None, 23)                207       
                                                                 
 dense_13 (Dense)            (None, 28)                672       
                                                                 
 dense_14 (Dense)            (None, 23)                667       
                                                                 
 dense_15 (Dense)            (None, 38)                912       
                                                                 
 dense_16 (Dense)            (None, 33)                1287      
                                                                 
 dense_17 (Dense)            (None, 1)                

In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics="accuracy")

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=20)

Epoch 1/20
804/804 [==============================] - 3s 2ms/step - loss: 0.6427 - accuracy: 0.6229
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5791 - accuracy: 0.7263
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5769 - accuracy: 0.7276
Epoch 4/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5762 - accuracy: 0.7278
Epoch 5/20
804/804 [==============================] - 3s 3ms/step - loss: 0.5755 - accuracy: 0.7283
Epoch 6/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5751 - accuracy: 0.7287
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5752 - accuracy: 0.7289
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5752 - accuracy: 0.7283
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5740 - accuracy: 0.7295
Epoch 10/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5735 - accuracy: 0.7292

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5544 - accuracy: 0.7331 - 680ms/epoch - 3ms/step
Loss: 0.5543721914291382, Accuracy: 0.7330612540245056


In [31]:
# Export our model to HDF5 file
nn.save_weights('AlphabetSoupCharity_Optimization_2.h5')

# Compiling, Training and Evaluating the Model: TRIAL 3/3

In [46]:
top_hyper[2].values

{'activation': 'tanh',
 'first_units': 23,
 'num_layers': 2,
 'units_0': 28,
 'units_1': 28,
 'units_2': 13,
 'units_3': 23,
 'units_4': 3,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [47]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[2].values['first_units'], activation = top_hyper[2].values['activation'], input_dim = num_of_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[2].values['units_0'], activation = top_hyper[2].values['activation']))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[2].values['units_1'], activation = top_hyper[2].values['activation']))

# Forth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[2].values['units_2'], activation = top_hyper[2].values['activation']))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[2].values['units_3'], activation = top_hyper[2].values['activation']))

# Sixth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[2].values['units_4'], activation = top_hyper[2].values['activation']))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = top_hyper[2].values['activation']))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 23)                805       
                                                                 
 dense_30 (Dense)            (None, 28)                672       
                                                                 
 dense_31 (Dense)            (None, 28)                812       
                                                                 
 dense_32 (Dense)            (None, 13)                377       
                                                                 
 dense_33 (Dense)            (None, 23)                322       
                                                                 
 dense_34 (Dense)            (None, 3)                 72        
                                                                 
 dense_35 (Dense)            (None, 1)                

In [48]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics="accuracy")

In [49]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=20)

Epoch 1/20
804/804 [==============================] - 4s 2ms/step - loss: 0.7813 - accuracy: 0.6943
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5832 - accuracy: 0.7203
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5839 - accuracy: 0.7157
Epoch 4/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5805 - accuracy: 0.7145
Epoch 5/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5700 - accuracy: 0.7201
Epoch 6/20
804/804 [==============================] - 3s 3ms/step - loss: 0.5589 - accuracy: 0.7275
Epoch 7/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5579 - accuracy: 0.7295
Epoch 8/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5637 - accuracy: 0.7236
Epoch 9/20
804/804 [==============================] - 3s 3ms/step - loss: 0.5633 - accuracy: 0.7287
Epoch 10/20
804/804 [==============================] - 3s 4ms/step - loss: 0.5563 - accuracy: 0.7300

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5482 - accuracy: 0.7315 - 689ms/epoch - 3ms/step
Loss: 0.548154354095459, Accuracy: 0.7315452098846436


In [37]:
# Export our model to HDF5 file
nn.save_weights('AlphabetSoupCharity_Optimization_3.h5')

# REPORT

# **Alphabet Soup Neural Network Model Analysis**

---

## **Overview of the Analysis**

The purpose of this analysis is to leverage machine learning, particularly deep learning neural networks, to predict the success of organizations seeking funding from the Alphabet Soup foundation. This tool will aid the foundation in optimizing its investments by funding organizations that are more likely to succeed in their ventures.


###**MODELING**

***Data Preprocessing***

- **Target Variable(s)**
  - The target for our model is **'IS_SUCCESSFUL'**.

- **Feature Variables**
  - The features considered for our model include:

    - 'APPLICATION_TYPE'
    - 'AFFILIATION'
    - 'CLASSIFICATION'
    - 'USE_CASE',
    - 'ORGANIZATION'
    - 'STATUS'
    - 'INCOME_AMT'
    - 'SPECIAL_CONSIDERATIONS'
    - 'ASK_AMT'
    - 'IS_SUCCESSFUL'

- **Variables to be Removed**
  - The following variables were removed from the input data since they neither served as targets nor features:
    - 'EIN'
    - 'NAME''
    

###***Compiling, Training, and Evaluating the Model***

**Neural Network Architecture**

  - **Rationale for Architecture Choice**:
    - Drop the non-beneficial ID columns `'EIN'` and `'NAME'`.
    - Choose a cutoff value and create a list of application types to be replaced `application_types_to_replace` and `classifications_to_replace`.
    - Convert categorical data to numeric with `pd.get_dummies`.
    - Split our preprocessed data into our features and target arrays.
    - Split the preprocessed data into a training and testing dataset.
    - Scale the data
    - Define de model as "neural network"
    

  - **Number of Neurons and Layers**:
    - The neural network model consists of `2` layers.
    - Layer 1: `10` neurons
    - Layer 2: `5` neurons

  - **Activation Functions**:
    - For input layer: `relu`
    - For hidden layers: `relu`
    - For output layer: `sigmoid`

- **Model Performance**
  - The model achieved a performance accuracy metric of `73.12%` which `IS_NOT` within the desired target of `75.00%`.


##**MODEL OPTIMIZATION**

###***Data Preprocessing***

- **Target Variable(s)**
  - The target for our model is **'IS_SUCCESSFUL'**.

- **Feature Variables**
  - The features considered for our model include:

    - 'APPLICATION_TYPE'
    - 'AFFILIATION'
    - 'CLASSIFICATION'
    - 'USE_CASE',
    - 'ORGANIZATION'
    - 'INCOME_AMT'
    - 'ASK_AMT'
    - 'IS_SUCCESSFUL'

- **Variables to be Removed**
  - The following variables were removed from the input data since they neither served as targets nor features:
    - 'STATUS'
    - 'SPECIAL_CONSIDERATIONS'

###***Compiling, Training, and Evaluating the Model***

**Neural Network Architecture**

  - **Rationale for Architecture Choice**:
    - Drop the non-beneficial ID columns `'STATUS'` and `'SPECIAL_CONSIDERATIONS'`.
    - Choose a cutoff value and create a list of application types to be replaced `affiliations_to_replace`, `use_cases_to_replace` and `organization_to_replace`.
    - Convert categorical data to numeric with `pd.get_dummies`.
    - Split our preprocessed data into our features and target arrays.
    - Split the preprocessed data into a training and testing dataset.
    - Scale the data.
    - Define de model as "neural network".
    - Run `Keras Tuner`.

- **Top 3 Keras Tuner Results**
  - **Top 1**: `{'activation': 'sigmoid', 'first_units': 33, 'units_0': 13, 'units_1': 3, 'units_2': 23, 'units_3': 18, 'units_4': 18, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0019'}`
  - **Top 2**: `{'activation': 'sigmoid', 'first_units': 8, 'units_0': 23, 'units_1': 28, 'units_2': 23, 'units_3': 38, 'units_4': 33, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0015'}`
  - **Top 3**: `{'activation': 'tanh', 'first_units': 23, 'units_0': 28, 'units_1': 28, 'units_2': 13, 'units_3': 23, 'units_4': 3, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}`

- **Model Performance**
  - **Top 1**: The model achieved a performance metric of `73.42%` which `IS_NOT` within the desired target of `75.00%`.
  - **Top 2**: The model achieved a performance metric of `73.31%` which `IS_NOT` within the desired target of `75.00%`.
  - **Top 3**: The model achieved a performance metric of `73.15%` which `IS_NOT` within the desired target of `75.00%`.


---

**CONCLUSION**

The developed neural network model showcases the potential of using `Keras Tuner` to optimize the success of prediction models. Even though `73.42%` `IS_NOT` within our initial target of `75.00%`, there was an increase of about `0.31%` compared to the original model.

---
